In [142]:
import geopandas
import pandas as pd
import geopandas as gpd
import folium

In [143]:
data = pd.read_csv('data/poland/RYNE_3786_CTAB_20250509085604.csv', sep=';')

In [144]:
data.head()

,Code,Name,total;total;2020;[PLN],total;total;2021;[PLN],total;total;2022;[PLN],total;total;2023;[PLN],total;total;2024;[PLN],total;to 40 m2;2020;[PLN],total;to 40 m2;2021;[PLN],total;to 40 m2;2022;[PLN],...,secondary market;from 60.1 to 80 m2;2021;[PLN],secondary market;from 60.1 to 80 m2;2022;[PLN],secondary market;from 60.1 to 80 m2;2023;[PLN],secondary market;from 60.1 to 80 m2;2024;[PLN],secondary market;from 80.1 m2;2020;[PLN],secondary market;from 80.1 m2;2021;[PLN],secondary market;from 80.1 m2;2022;[PLN],secondary market;from 80.1 m2;2023;[PLN],secondary market;from 80.1 m2;2024;[PLN],Unnamed: 77
0,0,POLAND,311467,348988,393524,427466,NaN,201152,226284,261041,...,388297,452500,512719,NaN,549670,607904,758603,829766,NaN,NaN
1,200000,DOLNOŚLĄSKIE,268487,340342,380369,419625,NaN,167974,222302,258875,...,352027,404170,463270,NaN,390941,467390,540474,616124,NaN,NaN
2,201000,Powiat bolesławiecki,223395,225740,248481,287441,NaN,116331,127699,168995,...,269723,289078,394540,NaN,304779,320632,345199,420925,NaN,NaN
3,202000,Powiat dzierżoniowski,115134,146681,156434,223186,NaN,73698,87044,102323,...,187085,202571,247649,NaN,202386,311299,236726,359145,NaN,NaN
4,203000,Powiat głogowski,215504,235327,263513,278506,NaN,132053,152369,155974,...,289564,303585,313409,NaN,352675,400239,431744,489586,NaN,NaN


In [145]:
new_df = data.iloc[:, [0, 1, 5]].rename(columns={"total;total;2023;[PLN]": "Price"})
new_df["Price Increase"] = (new_df["Price"]-data.iloc[:,2])/data.iloc[:,2]
new_df.head()

,Code,Name,Price,Price Increase
0,0,POLAND,427466,0.372428
1,200000,DOLNOŚLĄSKIE,419625,0.562925
2,201000,Powiat bolesławiecki,287441,0.286694
3,202000,Powiat dzierżoniowski,223186,0.938489
4,203000,Powiat głogowski,278506,0.292347


In [146]:
mapa_pow = gpd.read_file('data/poland/powiaty/powiaty.shp')
# ❗ Fix: reproject to EPSG:4326
mapa_pow = mapa_pow.to_crs(epsg=4326)
mapa_pow = mapa_pow[['JPT_KOD_JE', 'geometry']]

In [147]:
average_m2_gus = new_df.copy()

average_m2_gus['Code (Filtered)'] = average_m2_gus.Code.apply(lambda x: '0'+str(x) if len(str(x)) < 7 else str(x))
average_m2_gus['Code (Filtered)'] = average_m2_gus['Code (Filtered)'].apply(lambda s: s[:4])
average_m2_gus = average_m2_gus[average_m2_gus['Code'] != '0']
data_gus_pow = average_m2_gus[average_m2_gus['Code (Filtered)'].str[0:3] != '000']


In [148]:
data_gus_2 = data_gus_pow[data_gus_pow['Code (Filtered)'].str[0:2] != '00']
data_gus_2 = data_gus_2[data_gus_2['Code (Filtered)'].str[2:4] != '00']

In [149]:
data_gus_2['Price'] = pd.to_numeric(data_gus_2['Price'], errors='coerce')

In [150]:
mapa_pow['geometry'] = mapa_pow['geometry'].buffer(0)

In [151]:
mapa_pow.geometry = mapa_pow.geometry.simplify(0.001)
#mapa_pow['geometry'] = mapa_pow['geometry'].simplify(tolerance=0.001, preserve_topology=True)
pow_geoPath = mapa_pow.to_json()
mapa = folium.Map([52, 19], zoom_start=6)

chloro1 = folium.Choropleth(geo_data=pow_geoPath,  # GeoJSON
                  data=data_gus_2,
                  columns=['Code (Filtered)', 'Price'],
                  key_on='feature.properties.JPT_KOD_JE',  # key to GeoJSON
                  fill_color='YlOrRd',
                  fill_opacity=0.7,
                  line_opacity=0.2,
                  legend_name="Median Transaction Price")
chloro1.add_to(mapa)

In [152]:
m2 =mapa_pow.merge(data_gus_2[['Code (Filtered)', 'Price']], left_on='JPT_KOD_JE', right_on='Code (Filtered)', how='left')
m2["Price(EUR)"] = m2["Price"] / 4.22
folium.GeoJson(
    data=m2.to_json(),
    name='Price Tooltip',
    tooltip=folium.features.GeoJsonTooltip(
        fields=['Price', 'Price(EUR)'],
        aliases=['Average Price (PLN):', 'Average Price (EUR):'],
        localize=True
    ),
    style_function=lambda x: {
        'fillColor': 'transparent',
        'color': 'black',
        'weight': 0.5,
        'fillOpacity': 0.1,
    }
).add_to(mapa)

In [153]:
mapa.save(outfile="exports/map.html")

In [154]:
import numpy as np
data_gus_3 = data_gus_2.copy().replace([np.inf, -np.inf], np.nan)

In [155]:
mapa2 = folium.Map([52, 19], zoom_start=6)

folium.Choropleth(geo_data=pow_geoPath,  # GeoJSON
                  data=data_gus_3,
                  columns=['Code (Filtered)', 'Price Increase'],
                  key_on='feature.properties.JPT_KOD_JE',  # key to GeoJSON
                  fill_color='YlOrRd',
                  fill_opacity=0.7,
                  line_opacity=0.2,
                  legend_name="Median Prince Increase").add_to(mapa2)

In [156]:
data_gus_3

,Code,Name,Price,Price Increase,Code (Filtered)
2,201000,Powiat bolesławiecki,287441,0.286694,0201
3,202000,Powiat dzierżoniowski,223186,0.938489,0202
4,203000,Powiat głogowski,278506,0.292347,0203
5,204000,Powiat górowski,146458,0.408318,0204
6,205000,Powiat jaworski,281036,0.578304,0205
...,...,...,...,...,...
392,3217000,Powiat wałecki,233386,0.278561,3217
393,3218000,Powiat łobeski,200607,0.336186,3218
394,3261000,City with powiat status Koszalin,329357,NaN,3261
395,3262000,City with powiat status Szczecin,487867,0.223893,3262


In [157]:
m3 =mapa_pow.merge(data_gus_3, left_on='JPT_KOD_JE', right_on='Code (Filtered)', how='left')
#m3["Price(EUR)"] = m3["Price Increase"] / 4.22
m3["Price Increase"] = (m3["Price Increase"]*100).round(2)
folium.GeoJson(
    data=m3.to_json(),
    name='Price Tooltip',
    tooltip=folium.features.GeoJsonTooltip(
        fields=["Name", 'Price Increase'],#, 'Price(EUR)'],
        aliases=["Powiat Name",'Median Price Increase (PLN):'],#, 'Median Price Increase (EUR):'],
        localize=True
    ),
    style_function=lambda x: {
        'fillColor': 'transparent',
        'color': 'black',
        'weight': 0.5,
        'fillOpacity': 0.1,
    }
).add_to(mapa2)

In [158]:
mapa2.save(outfile="exports/map2.html")

In [159]:
df_fr = pd.read_csv("data/france/full.csv", sep=",", encoding="utf-8")

/var/folders/g7/bnv7sj75399d5qdncpcwkjzh0000gn/T/ipykernel_58627/2784223609.py:1: DtypeWarning: Columns (8,10,12,14,16,17,18,20,22,24,35,36) have mixed types. Specify dtype option on import or set low_memory=False.
  df_fr = pd.read_csv("data/france/full.csv", sep=",", encoding="utf-8")


In [160]:
df_fr.head()

,id_mutation,date_mutation,numero_disposition,nature_mutation,valeur_fonciere,adresse_numero,adresse_suffixe,adresse_nom_voie,adresse_code_voie,code_postal,...,type_local,surface_reelle_bati,nombre_pieces_principales,code_nature_culture,nature_culture,code_nature_culture_speciale,nature_culture_speciale,surface_terrain,longitude,latitude
0,2024-1,2024-01-02,1,Vente,346.5,NaN,NaN,LE DELIVRE,B020,1230.0,...,NaN,NaN,NaN,P,prés,NaN,NaN,99.0,5.530952,45.952439
1,2024-2,2024-01-03,2,Vente,10000.0,NaN,NaN,CHEVRY DESSOUS,B007,1170.0,...,NaN,NaN,NaN,S,sols,NaN,NaN,115.0,6.043339,46.282256
2,2024-3,2024-01-08,1,Vente,249000.0,NaN,NaN,PIN HAMEAU,B086,1290.0,...,NaN,NaN,NaN,S,sols,NaN,NaN,497.0,4.911143,46.247235
3,2024-4,2024-01-03,1,Vente,329500.0,9001.0,NaN,PL DU JURA,0500,1170.0,...,Dépendance,NaN,0.0,NaN,NaN,NaN,NaN,NaN,6.058695,46.332212
4,2024-4,2024-01-03,1,Vente,329500.0,29.0,NaN,PL DU JURA,0500,1170.0,...,Dépendance,NaN,0.0,NaN,NaN,NaN,NaN,NaN,6.058695,46.332212


In [161]:
df_fr.columns

Index(['id_mutation', 'date_mutation', 'numero_disposition', 'nature_mutation',
       'valeur_fonciere', 'adresse_numero', 'adresse_suffixe',
       'adresse_nom_voie', 'adresse_code_voie', 'code_postal', 'code_commune',
       'nom_commune', 'code_departement', 'ancien_code_commune',
       'ancien_nom_commune', 'id_parcelle', 'ancien_id_parcelle',
       'numero_volume', 'lot1_numero', 'lot1_surface_carrez', 'lot2_numero',
       'lot2_surface_carrez', 'lot3_numero', 'lot3_surface_carrez',
       'lot4_numero', 'lot4_surface_carrez', 'lot5_numero',
       'lot5_surface_carrez', 'nombre_lots', 'code_type_local', 'type_local',
       'surface_reelle_bati', 'nombre_pieces_principales',
       'code_nature_culture', 'nature_culture', 'code_nature_culture_speciale',
       'nature_culture_speciale', 'surface_terrain', 'longitude', 'latitude'],
      dtype='object')

In [162]:
len(df_fr)

3458643

In [163]:
df_fr.code_commune = df_fr.code_commune.apply(lambda x: str(x).zfill(5))

In [164]:
# Average by code_commune
df_fr = df_fr.groupby(['code_commune']).agg({'valeur_fonciere': 'median'}).reset_index()

In [165]:
df_fr.head()

,code_commune,valeur_fonciere
0,01001,350000.0
1,01002,25000.0
2,01004,180000.0
3,01005,243291.0
4,01006,10000.0


In [166]:
shp_fr = gpd.read_file('data/france/commune-frmetdrom/COMMUNE_FRMETDROM.shp')
# ❗ Fix: reproject to EPSG:4326
shp_fr = shp_fr.to_crs(epsg=4326)

In [167]:
shp_fr.columns

Index(['ID', 'NOM', 'NOM_M', 'INSEE_COM', 'STATUT', 'POPULATION', 'INSEE_CAN',
       'INSEE_ARR', 'INSEE_DEP', 'INSEE_REG', 'SIREN_EPCI', 'TYPE',
       'geometry'],
      dtype='object')

In [168]:
shp_fr.head()

,ID,NOM,NOM_M,INSEE_COM,STATUT,POPULATION,INSEE_CAN,INSEE_ARR,INSEE_DEP,INSEE_REG,SIREN_EPCI,TYPE,geometry
0,COMMUNE_0000000009747703,Antully,ANTULLY,71010,Commune simple,869.0,02,1,71,27,200070530,COM,"POLYGON ((4.42739 46.92275, 4.42745 46.92262, ..."
1,COMMUNE_0000000009734075,Le Tremblay-Omonville,LE TREMBLAY-OMONVILLE,27658,Commune simple,369.0,15,2,27,28,242700607,COM,"POLYGON ((0.90536 49.10568, 0.90375 49.10801, ..."
2,COMMUNE_0000000009730209,Vaux-lès-Mouzon,VAUX-LES-MOUZON,08466,Commune simple,69.0,03,3,08,44,240800847,COM,"POLYGON ((5.1528 49.57652, 5.15277 49.57688, 5..."
3,COMMUNE_0000000009742448,Anchenoncourt-et-Chazel,ANCHENONCOURT-ET-CHAZEL,70017,Commune simple,240.0,11,2,70,27,200041879,COM,"POLYGON ((6.14568 47.84562, 6.14458 47.84435, ..."
4,COMMUNE_0000002324290054,Culoz-Béon,CULOZ-BEON,01138,Commune simple,3426.0,10,1,01,84,200040350,COM,"POLYGON ((5.80753 45.86064, 5.80739 45.86051, ..."


In [169]:
shp_fr = shp_fr[['INSEE_COM', 'geometry']]
shp_fr['INSEE_COM_numeric'] = pd.to_numeric(shp_fr['INSEE_COM'], errors='coerce')

# Drop rows where conversion failed (i.e., not numeric)
shp_fr = shp_fr[shp_fr['INSEE_COM_numeric'].notna()]

# Convert to integer type and drop helper column
shp_fr['INSEE_COM'] = shp_fr['INSEE_COM_numeric'].apply(lambda x: str(int(x)).zfill(5))
shp_fr = shp_fr.drop(columns='INSEE_COM_numeric')
shp_fr.head()

,INSEE_COM,geometry
0,71010,"POLYGON ((4.42739 46.92275, 4.42745 46.92262, ..."
1,27658,"POLYGON ((0.90536 49.10568, 0.90375 49.10801, ..."
2,08466,"POLYGON ((5.1528 49.57652, 5.15277 49.57688, 5..."
3,70017,"POLYGON ((6.14568 47.84562, 6.14458 47.84435, ..."
4,01138,"POLYGON ((5.80753 45.86064, 5.80739 45.86051, ..."


In [170]:
shp_fr['geometry'] = shp_fr['geometry'].buffer(0)

In [171]:


sensible_max_for_detailed_legend = 1000000
my_bins = [0, 50000, 100000, 150000, 250000, 500000, 1000000]#, df_fr['valeur_fonciere'].max()]
df_fr['valeur_fonciere_capped'] = df_fr['valeur_fonciere'].clip(upper=sensible_max_for_detailed_legend)
# my_bins = list(df_fr['valeur_fonciere'][df_fr['valeur_fonciere'] <= sensible_max_for_detailed_legend].quantile([0, 0.2, 0.4, 0.6, 0.8]))
# my_bins.append(sensible_max_for_detailed_legend) # Add the threshold for "high"
# my_bins.append(df_fr['valeur_fonciere'].max()) # Ensure all values are covered

# # Remove duplicates and sort, as quantile might produce them if data is sparse
# my_bins = sorted(list(set(my_bins)))

# # Ensure the lowest bin starts at or below your data's minimum if necessary, or 0.
# if my_bins[0] > 0 and df_fr['valeur_fonciere'].min() < my_bins[0]:
#     my_bins.insert(0, df_fr['valeur_fonciere'].min() if df_fr['valeur_fonciere'].min() < 0 else 0) # or just 0
#     my_bins = sorted(list(set(my_bins))) # re-sort and unique


In [172]:
shp_fr.geometry = shp_fr.geometry.simplify(0.005)
pow_geoPath = shp_fr.to_json()
map3 = folium.Map([52, 19], zoom_start=6)


folium.Choropleth(geo_data=pow_geoPath,  # GeoJSON
                  data=df_fr,
                  columns=['code_commune', 'valeur_fonciere_capped'],
                  key_on='feature.properties.INSEE_COM',  # key to GeoJSON
                  fill_color='YlOrRd',
                  fill_opacity=0.7,
                  line_opacity=0.2,
                  legend_name="Mean Transaction Price",
                    bins=my_bins,  # This is the crucial part!
                    highlight=True,
                    smooth_factor=0 # If you simplified geometries, this can help render faster
                    ).add_to(map3)

In [173]:
chloro1.add_to(map3)

In [174]:
folium.LayerControl().add_to(map3)

In [175]:
map3.save(outfile="exports/map_fr.html")

In [217]:
shp_fr_dep = gpd.read_file('data/france/contour-des-departements.geojson')
shp_fr_dep = shp_fr_dep.to_crs(epsg=4326)
shp_fr_dep['code_numeric'] = pd.to_numeric(shp_fr_dep['code'], errors='coerce')
shp_fr_dep = shp_fr_dep[shp_fr_dep['code_numeric'].notna()]
shp_fr_dep['code'] = shp_fr_dep['code'].apply(lambda x: str(x).zfill(3))
shp_fr_dep = shp_fr_dep.drop(columns='code_numeric')

df_fr_dep = pd.read_csv("data/france/full.csv", sep=",", encoding="utf-8")
df_fr_dep['code_postal'] = df_fr_dep['code_postal'].apply(lambda x: str(x).zfill(5))
df_fr_dep['code_dep'] = df_fr_dep['code_postal'].apply(lambda x: str(x)[:2] if str(x)[:2] not in ["97", "98"] else str(x)[:3])
df_fr_dep['code_dep'] = df_fr_dep['code_dep'].apply(lambda x: str(x).zfill(3))
df_fr_dep.dropna(inplace=True, subset=['valeur_fonciere'])
df_fr_dep = df_fr_dep.groupby(['code_dep']).agg({'valeur_fonciere': 'median'}).reset_index()

/var/folders/g7/bnv7sj75399d5qdncpcwkjzh0000gn/T/ipykernel_58627/1373090204.py:8: DtypeWarning: Columns (8,10,12,14,16,17,18,20,22,24,35,36) have mixed types. Specify dtype option on import or set low_memory=False.
  df_fr_dep = pd.read_csv("data/france/full.csv", sep=",", encoding="utf-8")


In [218]:
df_fr_dep.columns, data_gus_2.columns

(Index(['code_dep', 'valeur_fonciere'], dtype='object'),
 Index(['Code', 'Name', 'Price', 'Price Increase', 'Code (Filtered)'], dtype='object'))

In [219]:
shp_fr_dep.columns, mapa_pow.columns

(Index(['code', 'nom', 'geometry'], dtype='object'),
 Index(['JPT_KOD_JE', 'geometry'], dtype='object'))

In [249]:
# Rename columns to match
## Codes
df_fr_dep.rename(columns={'code_dep': 'eurocode'}, inplace=True)
shp_fr_dep.rename(columns={'code': 'eurocode'}, inplace=True)
df_pl = data_gus_2.rename(columns={'Code (Filtered)': 'eurocode'})
shp_pl = mapa_pow.rename(columns={'JPT_KOD_JE': 'eurocode'})
## Prices
df_fr_dep.rename(columns={'valeur_fonciere': 'price'}, inplace=True)
df_pl.rename(columns={'Price': 'price'}, inplace=True)

# Select identical columns for merging
shp_fr_dep = shp_fr_dep[['eurocode', 'geometry']]
shp_pl = shp_pl[['eurocode', 'geometry']]
df_fr_dep = df_fr_dep[['eurocode', 'price']]
df_pl = df_pl[['eurocode', 'price']]

# Convert to currency
df_pl['price'] = df_pl['price'].apply(lambda x: x / 4.22)

# Standardize to eurocode format : code starts with two country letter code
df_fr_dep['eurocode'] = 'FR' + df_fr_dep['eurocode']
shp_fr_dep['eurocode'] = 'FR' + shp_fr_dep['eurocode']
df_pl['eurocode'] = 'PL' + df_pl['eurocode']
shp_pl['eurocode'] = 'PL' + shp_pl['eurocode']


# Merge the dataframes
eurodf = pd.merge(df_fr_dep, df_pl, on=['eurocode', 'price'], how='outer')
euroshp = pd.merge(shp_fr_dep, shp_pl, on=['eurocode','geometry'], how='outer')

In [250]:
eurodf.head()

,eurocode,price
0,FR001,168000.000
1,FR002,88800.000
2,FR003,82500.000
3,FR004,110000.000
4,FR005,110390.235


In [251]:

euroshp.head()

,eurocode,geometry
0,FR001,"POLYGON ((4.78021 46.17668, 4.78024 46.18905, ..."
1,FR002,"POLYGON ((3.17296 50.01131, 3.17382 50.01186, ..."
2,FR003,"POLYGON ((3.03207 46.79491, 3.03424 46.7908, 3..."
3,FR004,"POLYGON ((5.67604 44.19143, 5.67817 44.19051, ..."
4,FR005,"POLYGON ((6.26057 45.12685, 6.26417 45.12641, ..."


In [252]:
euroshp.geometry = euroshp.geometry.simplify(0.001)
pow_geoPath = euroshp.to_json()
map5 = folium.Map([50, 10], zoom_start=5)


folium.Choropleth(geo_data=pow_geoPath,  # GeoJSON
                  data=eurodf,
                  columns=['eurocode', 'price'],#_capped'],
                  key_on='feature.properties.eurocode',  # key to GeoJSON
                  fill_color='YlOrRd',
                  fill_opacity=0.7,
                  line_opacity=0.2,
                  legend_name="Mean Transaction Price",
                    #bins=my_bins,  # This is the crucial part!
                    highlight=True,
                    smooth_factor=0 # If you simplified geometries, this can help render faster
                    ).add_to(map5)

euromerge = euroshp.merge(eurodf[['eurocode', 'price']], left_on='eurocode', right_on='eurocode', how='left')
folium.GeoJson(
    data=euromerge.to_json(),
    name='Price Tooltip',
    tooltip=folium.features.GeoJsonTooltip(
        fields=['price'],
        aliases=['Median Price (EUR):'],
        localize=True
    ),
    style_function=lambda x: {
        'fillColor': 'transparent',
        'color': 'black',
        'weight': 0.5,
        'fillOpacity': 0.1,
    }
).add_to(map5)


map5.save(outfile="exports/euromap.html")

In [253]:
df_fr_dep['eurocode'].unique()

array(['FR001', 'FR002', 'FR003', 'FR004', 'FR005', 'FR006', 'FR007',
       'FR008', 'FR009', 'FR010', 'FR011', 'FR012', 'FR013', 'FR014',
       'FR015', 'FR016', 'FR017', 'FR018', 'FR019', 'FR020', 'FR021',
       'FR022', 'FR023', 'FR024', 'FR025', 'FR026', 'FR027', 'FR028',
       'FR029', 'FR030', 'FR031', 'FR032', 'FR033', 'FR034', 'FR035',
       'FR036', 'FR037', 'FR038', 'FR039', 'FR040', 'FR041', 'FR042',
       'FR043', 'FR044', 'FR045', 'FR046', 'FR047', 'FR048', 'FR049',
       'FR050', 'FR051', 'FR052', 'FR053', 'FR054', 'FR055', 'FR056',
       'FR058', 'FR059', 'FR060', 'FR061', 'FR062', 'FR063', 'FR064',
       'FR065', 'FR066', 'FR069', 'FR070', 'FR071', 'FR072', 'FR073',
       'FR074', 'FR075', 'FR076', 'FR077', 'FR078', 'FR079', 'FR080',
       'FR081', 'FR082', 'FR083', 'FR084', 'FR085', 'FR086', 'FR087',
       'FR088', 'FR089', 'FR090', 'FR091', 'FR092', 'FR093', 'FR094',
       'FR095', 'FR971', 'FR972', 'FR973', 'FR974'], dtype=object)

In [226]:
len(df_fr_dep['eurocode'].unique()  )

94

In [246]:
shp_fr_dep = gpd.read_file('data/france/contour-des-departements.geojson')
shp_fr_dep = shp_fr_dep.to_crs(epsg=4326)
shp_fr_dep['code_numeric'] = pd.to_numeric(shp_fr_dep['code'], errors='coerce')
shp_fr_dep = shp_fr_dep[shp_fr_dep['code_numeric'].notna()]
shp_fr_dep['code'] = shp_fr_dep['code'].apply(lambda x: str(x).zfill(3))
shp_fr_dep = shp_fr_dep.drop(columns='code_numeric')

df_fr_dep = pd.read_csv("data/france/full.csv", sep=",", encoding="utf-8")



# Step 1: Coerce everything to numeric, invalid ones become NaN
df_fr_dep['code_postal'] = pd.to_numeric(df_fr_dep['code_postal'], errors='coerce')

# Step 2: Drop or handle rows where postal code is NaN
# Option A: Drop them
df_fr_dep = df_fr_dep.dropna(subset=['code_postal'])

# Step 3: Convert to int, then to string with leading zeros
df_fr_dep['code_postal'] = df_fr_dep['code_postal'].astype(int).astype(str).str.zfill(5)



df_fr_dep['code_dep'] = df_fr_dep['code_postal'].apply(lambda x: str(x)[:2] if str(x)[:2] not in ["97", "98"] else str(x)[:3])
df_fr_dep['code_dep'] = df_fr_dep['code_dep'].apply(lambda x: str(x).zfill(3))
df_fr_dep.dropna(inplace=True, subset=['valeur_fonciere'])
df_fr_dep = df_fr_dep.groupby(['code_dep']).agg({'valeur_fonciere': 'median'}).reset_index()

/var/folders/g7/bnv7sj75399d5qdncpcwkjzh0000gn/T/ipykernel_58627/701886070.py:8: DtypeWarning: Columns (8,10,12,14,16,17,18,20,22,24,35,36) have mixed types. Specify dtype option on import or set low_memory=False.
  df_fr_dep = pd.read_csv("data/france/full.csv", sep=",", encoding="utf-8")
